# Ceneo Webscrapper

# Struktura pojedynczej opinii w serwisie Ceneo.pl

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii| ["data-entry-id"] | opinion_id |
|autor| span.user-post__author-name | author |
|rekomendacja| span.user-post__author-recomendation > em | recommendation |
|gwiazdki| span.user-post__score-count | stars |
|treść| div.user-post__text | content |
|lista zalet| div.review-feature__title--positives ~ div.review-feature__item | pros |
|lista wad| div.review-feature__title--negatives ~ div.review-feature__item | cons |
|dal ilu przydatna| span[id^="votes-yes"] | useful |
|dla ilu nieprzydatna| span[id=^"votes-no"] | useless |
|data wystawienia| user-post__published > time:nth-child(1)["datetime"]| post_date |
|data zakupu| user-post__published > time:nth-child(2)["datetime"] | purchace_date |

# Biblioteki

In [6]:
import os
import json
import requests
from bs4 import BeautifulSoup

# Zadanie 1

# Wysłanie requesta

In [7]:
product_id = "114700014"
url = f"https://www.ceneo.pl/{product_id}/opinie-2"



In [8]:
all_opinions = []
while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, 'html.parser')
    opinions = page_dom.select("div.js_product-review")
    opinion = page_dom.select_one("div.js_product-review")

    for opinion in opinions:
        try:
            single_opinion = {
                "opinion_id": opinion["data-entry-id"],
                "author": opinion.select_one("span.user-post__author-name").get_text().strip(),
                "recommendation": opinion.select_one("span.user-post__author-recomendation > em").get_text().strip(),
                "stars": opinion.select_one("span.user-post__score-count").get_text().strip(),
                "content": opinion.select_one("div.user-post__text").get_text().strip(),
                "pros": [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
                "cons": [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
                "useful": opinion.select_one("span[id^='votes-yes']").get_text().strip(),
                "useless": opinion.select_one("span[id^='votes-no']").get_text().strip(),
                "purchased": opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
                "publish_date": opinion.select_one("span.user-post__published > time:nth-child(2)")['datetime'].strip()
            }
            all_opinions.append(single_opinion)
        except (AttributeError, TypeError):
            pass    
    try:    
        url = "https://www.ceneo.pl/"+page_dom.select_one("a.pagination__next")['href']
    except TypeError:
        url = None

In [9]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)


In [10]:
print(len(all_opinions))
print(all_opinions)

18
[{'opinion_id': '18226826', 'author': 'a...i', 'recommendation': 'Polecam', 'stars': '5/5', 'content': 'To rzeczywiście wysoka półka.\nJedyną wadą jest chęć producenta do uproszczenia obsługi - poszedł w tym za bardzo w kierunku "jednoprzyciskowości" - lepsze jest wrogiem dobrego, ale żeby to pojąć trzeba dojrzeć. Niemniej i tak jestem bardzo zadowolony. Stary odkurzacz jest już w piwnicy.', 'pros': ['dużo końcówek', 'lekki', 'poręczny', 'przycisk zasilania', 'wyświetlacz'], 'cons': ['słaba bateria'], 'useful': '2', 'useless': '0', 'purchased': '2023-12-20 00:33:01', 'publish_date': '2023-12-08 22:22:49'}, {'opinion_id': '15437754', 'author': 'nemesis wawa', 'recommendation': 'Polecam', 'stars': '5/5', 'content': 'Zmiana z v8 zdecydowanie na plus. Dużo lżejszy i zwinniejszy. Opcja pokazywania czasu pozostałego do końca pracy baterii bardzo przydatna. Nie trzeba trzymać przycisku włączania.. Solidne wykonanie. Wart swojej ceny :)', 'pros': ['dużo końcówek', 'lekki', 'poręczny', 'przy